In [1]:
import numpy as np
import sys
sys.path.append('../src/PrimeFactorization/')
from PrimeFactorization import primeFactorize
from tqdm.auto import tqdm
import copy
import os
from Levenshtein import distance as levenshtein_distance
import pandas as pd
import json
from transformers import get_linear_schedule_with_warmup

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pprint

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [42]:
t = tokenizer.Tokenizer()

In [3]:
data = {}

In [4]:
max_power = 16
# 2*x power to accomodate for 100=10*10 + 1x power - 1 for multiplication symbols + 1 for end of sequence
max_input_size = max_power+1
max_decode_size = 3*max_power

In [5]:
pbar = tqdm(total = 2**max_power - 2)
for i in range(2, 2**max_power):
    data[i] = primeFactorize(i)
    pbar.update(1)

In [6]:
len(data)

65534

In [ ]:
global_factor_mapping = copy.deepcopy(data)

In [ ]:
def is_prime(number):
    try:
        factored = global_factor_mapping[number]
        return len(factored)==1 and factored[list(factored.keys())[0]]==1
    except KeyError:
        return None

In [7]:
train_data = {}
test_data = {}
train_counter = 0
test_counter = 0

train_prob = .8
np.random.seed(0)
for number, factor in data.items():
    if np.random.rand() < train_prob:
        train_data[train_counter] = {'number' : number, 'factors' : factor} 
        train_counter +=1
    else:
        test_data[test_counter] = {'number' : number, 'factors' : factor} 
        test_counter +=1


In [8]:
len(train_data), len(test_data)

(52517, 13017)

In [ ]:
data = binarize_data(data)

In [ ]:
train_data = {}
test_data = {}
train_counter = 0
test_counter = 0

train_prob = .8
np.random.seed(0)
for number, factor in data.items():
    if np.random.rand() < train_prob:
        train_data[train_counter] = {'number' : number, 'factors' : factor} 
        train_counter +=1
    else:
        test_data[test_counter] = {'number' : number, 'factors' : factor} 
        test_counter +=1


In [ ]:
train_dataset = FactorizationDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = FactorizationDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
train_dataset.data_dict[5]

In [ ]:
class PositionalEncoding(nn.Module):
    # From: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    def __init__(self, d_model, dropout, max_len):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
class TransformerEmbedding(nn.Module):
    def __init__(self, n_tokens, embed_dim, dropout):
        super(TransformerEmbedding, self).__init__()
        self.embedding = nn.Embedding(n_tokens, embed_dim)
        self.pe = PositionalEncoding(embed_dim, dropout, max_decode_size)
        
    def forward(self, x):
        x = self.embedding(x).transpose(0,1)
        x = self.pe(x)
        return x

## Waht other junk goes on when  training a transfoerm? learning rate schedulign?
## MORE DATA
# optimize Decode
# Harden --> github
# Increase bathc size
# Gradient clipping can too
# Make entireley configurable based on a config
# Gradietn accumulation
# Script to generate data
# evaluation scrip9t
# config should have an overwrite function. Clears directory if yes. Otherwise break
# Deal with stupid bug in metrics......


In [ ]:
class Factorizer(nn.Module):
    def __init__(self, n_tokens, embed_dim, shared_embeddings, **kwargs):
        super(Factorizer, self).__init__()
        self.shared_embeddings = shared_embeddings
        dropout = .1 if not 'dropout' in kwargs else kwargs['dropout']
        if not shared_embeddings:
            self.src_embedding = TransformerEmbedding(n_tokens, embed_dim, dropout)
            self.tgt_embedding = TransformerEmbedding(n_tokens, embed_dim, dropout)
        else:
            self.embedding = TransformerEmbedding(n_tokens, embed_dim, dropout)
        
        self.transformer = nn.Transformer(d_model = embed_dim, **kwargs)
        
        self.tokens_out = nn.Linear(embed_dim, n_tokens)
        
    def form_pad_mask(self, tokens):
        return (tokens==tokenizer('_')[0]).to(tokens.device)
    
    def form_subsequence_mask(self, tgt):
        size = tgt.size(0)
        return (torch.triu(torch.ones(size, size)) == 0).transpose(0,1).to(tgt.device)
    
    def route_embeddings(self, src_or_tgt, input_type):
        if self.shared_embeddings:
            return self.embedding(src_or_tgt)
        if input_type=='src':
            return self.src_embedding(src_or_tgt)
        elif input_type=='tgt':
            return self.tgt_embedding(src_or_tgt)
    
    def encode(self, src):
        src_key_padding_mask = self.form_pad_mask(src)
        src = self.route_embeddings(src, 'src')
        
        return self.transformer.encoder(src, src_key_padding_mask = src_key_padding_mask), src_key_padding_mask
    
    def decode(self, tgt, memory, memory_key_padding_mask):
        tgt_key_padding_mask = self.form_pad_mask(tgt)
        tgt = self.route_embeddings(tgt, 'tgt')
        
        tgt_mask = self.form_subsequence_mask(tgt)
        
        output = self.transformer.decoder(tgt, memory, tgt_mask = tgt_mask, 
                                          tgt_key_padding_mask = tgt_key_padding_mask,
                                          memory_key_padding_mask = memory_key_padding_mask)
        output = output.transpose(0,1)
        decoded = self.tokens_out(output)
        return decoded
    
        
    def forward(self, src, tgt):
        memory, memory_key_padding_mask = self.encode(src)
        decoded = self.decode(tgt, memory, memory_key_padding_mask)
        return decoded

In [ ]:
device = torch.device('cuda')

In [ ]:
model_parameters = {
    'n_tokens' : len(tokenizer),
    'embed_dim' : 64,
    'num_encoder_layers' : 8,
    'num_decoder_layers' : 8,
    'dim_feedforward' : 128,
    'dropout' : .1,
    'shared_embeddings' : True,
}
opt_args = {
    'n_warmup_steps' : 2000,
    'nb_epochs' : 50,
}
opt_args['total_steps'] = opt_args['nb_epochs'] * len(train_loader)

In [ ]:
save_path = '../models/pad_instead_of_zero/'
os.makedirs(save_path, exist_ok=True)

In [ ]:
def get_best_model_file(path):
    files = [f for f in os.listdir(path) if f.endswith('.pt')]
    original_files = files
    files = [f.split('_')[1] for f in files]
    losses = [float('.'.join(f.split('.')[:-1])) for f in files]
    try:
        best_loss_idx = np.argmin(losses)
    except ValueError:
        return None
    return path + original_files[best_loss_idx]

In [ ]:
tokenizer = Tokenizer()

In [ ]:
# checkpoint_path = save_path + "51_0.1157.pt"
checkpoint_path = get_best_model_file(save_path)
# checkpoint_path = None
checkpoint_path

In [ ]:
if checkpoint_path:
    checkpoint = torch.load(checkpoint_path)
    model = Factorizer(**checkpoint['model_parameters'])
    model.load_state_dict(checkpoint['state_dict'])
    opt = optim.Adam(model.parameters())
    opt.load_state_dict(checkpoint['opt'])
    model.to(device)
#     scheduler = get_linear_schedule_with_warmup(opt, checkpoint['opt_args']['n_warmup_steps'], checkpoint['opt_args']['total_steps'], last_epoch=checkpoint['opt']['epoch'])
else:
    model = Factorizer(**model_parameters).to(device)
    opt = optim.Adam(model.parameters())
    scheduler = get_linear_schedule_with_warmup(opt, opt_args['n_warmup_steps'], opt_args['total_steps'])
loss_func = nn.CrossEntropyLoss()

In [ ]:
def run_batch(batch, backward=True):
    numbers = torch.tensor(tokenizer(batch['number'])).to(device)
    labels = torch.tensor(tokenizer(batch['label'])).to(device)
    res = model(numbers, labels[:,:-1])
    loss = loss_func(res.view(-1, 5), labels[:,1:].reshape(-1))
    if backward:
        loss.backward()
    return loss.item()
    

In [8]:
test[123]

{3: 1, 41: 1}

In [10]:
test.is_prime(69)

False

In [ ]:
def train_for_epoch(loader):
    model.train()
    pbar = tqdm(total = len(loader), leave=False)
    loss_hist = []
    for i, batch in enumerate(loader):
        model.zero_grad()
        loss = run_batch(batch, backward=True)
        opt.step()
        scheduler.step()
        pbar.update(1)
        loss_hist.append(loss)
    pbar.close()
    mean_loss = np.mean(loss_hist)
    
    return mean_loss

In [ ]:
def test_for_epoch(loader):
    model.eval()
    pbar = tqdm(total = len(loader), leave=False)
    loss_hist = []
    with torch.no_grad():
        for i, batch in enumerate(loader):
            loss_hist.append(run_batch(batch, backward=False))
            pbar.update(1)
    pbar.close()
    return np.mean(loss_hist)

In [ ]:
# pbar = tqdm(total = opt_args['nb_epochs'])
# for i in range(opt_args['nb_epochs'] ):
#     train_loss = train_for_epoch(train_loader)
#     test_loss = test_for_epoch(test_loader)
    
#     if save_path:
#         torch.save({'epoch' : i, 
#                     'state_dict' : model.state_dict(), 
#                     'opt': opt.state_dict(), 
#                     'train_loss' : train_loss, 
#                     'test_loss' : test_loss,
#                     'model_parameters' : model_parameters,
#                     'opt_args': opt_args,
#                     'epoch' : i},
#                     '%s/%d_%.4f.pt'%(save_path, i, test_loss))
    
    
    
    
    
    
#     print('Train: %.6f, Test: %.6f'%(train_loss, test_loss))
#     pbar.update(1)

In [ ]:
def decode(memory, memory_key_padding_mask, n_beams=2):
    sequences = torch.tensor(tokenizer('1')).to(device).unsqueeze(0)
    sequence_log_probs = torch.tensor([[0]]).to(device)
    stop_char = tokenizer('.')[0]
    
    for i in range(max_decode_size-1):
        with torch.no_grad():
            output = model.decode(sequences, 
                                  memory.repeat(1,sequences.size(0),1), 
                                  memory_key_padding_mask.repeat(sequences.size(0), 1))[:,-1]
#                                   memory_key_padding_mask)[:,-1]
            output = torch.log_softmax(output, dim=-1)
        
        # Compue all possible next sequences
        sequences = torch.repeat_interleave(sequences, len(tokenizer), 0)
        next_tokens = torch.arange(len(tokenizer)).repeat(output.size(0)).unsqueeze(1).to(sequences.device)
        next_sequences = torch.cat((sequences, next_tokens), dim=1)
        
        # Get the indiclies of the highest probability sequences
        next_token_log_probs = output.view(-1,1)
        # For cases when the model migth predict a non padding token after an end of sequence token
        # Manually set the log probability to be very low so it's never chosen to be decdoed
        # Additionally, break if all sequences have eos tokens
        seq_has_eos = torch.argmax((sequences==tokenizer('.')[0]).int(), dim=1).view(-1,1) > 0
        if torch.all(seq_has_eos):
            # undo repeat_interleave
            sequences = sequences[::len(tokenizer)]
            break

        # Compute all possible next log probabilities
        next_token_log_probs[seq_has_eos & ~(next_tokens==tokenizer('_')[0])] = -np.log(10000)
        sequence_log_probs = torch.repeat_interleave(sequence_log_probs, len(tokenizer), 0)
        next_sequence_log_probs = sequence_log_probs + next_token_log_probs
        
        top_indicies = torch.argsort(next_sequence_log_probs, dim=0, descending=True).squeeze()[:n_beams]
        
        sequences = next_sequences[top_indicies]
        sequence_log_probs = next_sequence_log_probs[top_indicies]
    
    sequences = sequences.data.cpu().numpy()
    sequence_log_probs = sequence_log_probs.data.cpu().numpy()
    return sequences, sequence_log_probs


def compute_full_target_str(base_10_number):
    number = dec2bin(base_10_number)
    factors = {dec2bin(k) : v for k, v in global_factor_mapping[base_10_number].items()}
    tmp_ds = FactorizationDataset({0:{'number' : number, 'factors' : factors}})
    return tmp_ds[0]['label']
        
def postprocess(factor_list, log_prob, base_10_number, number, beam_idx):
    tokenized = tokenizer(factor_list)
    
    information = {
        'target_num' : base_10_number,
        'target_is_prime' : is_prime(base_10_number),
        'target_str_full' : compute_full_target_str(base_10_number),
        'target_factor_list' : sum([[k]*v for k, v in global_factor_mapping[base_10_number].items()], []),
        'pred_str_full' : ''.join(tokenizer(factor_list, decode_special=True)) + '_'*(max_decode_size-len(factor_list)),
        'pred_str' : ''.join(tokenized),
        'beam_idx' : beam_idx,
        'log_prob' : log_prob.item(),
    }
    information['n_target_factors'] = len(information['target_factor_list'])
    
    factor_list = tokenized.split('x')
    try:
        factors = [bin2dec(num) for num in factor_list]
    except ValueError:
        factors = []
    
    information['pred_factor_list'] = factors
    information['n_pred_factors'] = len(information['pred_factor_list'])
    if len(information['pred_factor_list']) > 0:
        information['product'] = np.prod(factors)
    else:
        information['product'] = np.nan
    information['correct_product'] = information['product']==base_10_number
    information['correct_factorization'] = information['correct_product'] & all([is_prime(n) for n in information['pred_factor_list']])
    
    information['num_prime_factors_pred'] = np.sum([len(global_factor_mapping[f])==1 for f in factors if f in global_factor_mapping])
    information['percent_prime_factors_pred'] = information['num_prime_factors_pred'] / information['n_pred_factors']
        
    
    target_str_full = information['target_str_full']
    pred_str_full = information['pred_str_full']
    information['seq_dist_binary'] = 1-np.mean([c1==c2 for c1, c2 in zip(target_str_full, pred_str_full)])
    information['seq_dist_lev'] = levenshtein_distance(target_str_full, pred_str_full)
    return information
    

        
def factor(number, n_beams=1, return_type='df'):
    base_10_num = number
    model.eval()
    
    # Conver the number to a tensor
    number = dec2bin(number)
    number = tokenizer(pad_input(number))
    number = torch.tensor(number).unsqueeze(0).to(device)
    # Encode the number
    with torch.no_grad():
        memory, memory_key_padding_mask = model.encode(number)
    # Decode!
    factors_list, log_probs = decode(memory, memory_key_padding_mask, n_beams)
    number = number.data.cpu().numpy()[0]
    to_return = []
    for i in range(len(factors_list)):
        to_return.append(postprocess(factors_list[i], log_probs[i], base_10_num, number, i))
        
    if return_type=='df':
        return pd.DataFrame.from_dict(to_return)
    elif return_type=='dict':
        return to_return
    else:
        raise ValueError('got unexpected return type %s'%return_type)
  

In [ ]:
def form_metric_df(items, n_beams=1, max_num=-1):
    rows = []
    
    pbar = tqdm(total = min(len(items), max_num) if max_num > 0 else len(items), leave=False)
    for i, num in enumerate(items):
        if max_num > 0 and i >= max_num:
            break
        if num < 2: continue
        rows.append(factor(num, n_beams, return_type='dict'))
        pbar.update(1)
    pbar.update(2)
    pbar.close()
    rows = sum(rows, [])
    df = pd.DataFrame.from_dict(rows)
    return df

In [ ]:
factor(45165, n_beams=5)

In [ ]:
factor(1221, n_beams=5)

In [ ]:
factor(127, n_beams=5)

In [ ]:
factor(456, n_beams=5)

In [ ]:
factor(4, n_beams=2)

In [ ]:
test_numbers  = []
for k, v in test_dataset.data_dict.items():
    test_numbers.append(bin2dec(v['number']))
len(test_numbers)

In [ ]:
model.load_state_dict(torch.load(get_best_model_file(save_path))['state_dict'])
model.eval()

In [ ]:
metric_df = form_metric_df(test_numbers, n_beams=3)

In [ ]:
metric_df.to_pickle(save_path + 'metric_df.pkl')

In [ ]:
metric_df.shape

In [ ]:
def compute_metrics(metric_df):
    metrics = {}
    grouped_by_num = metric_df.groupby('target_num')
    metrics['correct'] = grouped_by_num.agg({
        'correct_product' : 'any',
        'correct_factorization' : 'any'
    }).mean(axis=0).to_dict()
    
    metrics['beam_accuracy'] = metric_df.groupby('beam_idx').agg({
        'correct_product' : 'mean',
        'correct_factorization' : 'mean'
    }).to_dict()
    
    metric_df['log_prob_decile'] = pd.qcut(metric_df['log_prob'], q=10).apply(str)
    metrics['by_prob'] = metric_df.groupby('log_prob_decile').agg({
        'correct_product': 'mean',
        'correct_factorization' : 'mean',
        'seq_dist_binary' : 'mean',
        'seq_dist_lev' : 'mean',
        'percent_prime_factors_pred' : 'mean',
    }).to_dict()
    
    
    # Things about the target, we want to take the first one of. Otherwise, we want all of them
    grouped_by_num = grouped_by_num.agg({k: 'first' if 'target' in k else list for k in list(metric_df) if not k=='target_num'}).reset_index()
    
    metrics['by_n_target_factors'] = grouped_by_num.groupby('n_target_factors').agg({
        'correct_product' : [lambda x: pd.Series([np.mean([any(y) for y in x])]), 'size'],
        'correct_factorization' : [lambda x: pd.Series([np.mean([any(y) for y in x])]), 'size']
    })
    metrics['by_n_target_factors'].columns = ['correct_product_mean', 'correct_product_size', 'correct_factorization_mean', 'correct_factorization_size']
    metrics['by_n_target_factors'] = metrics['by_n_target_factors'].to_dict()
    
    grouped_by_num['number_decile'] = pd.qcut(grouped_by_num['target_num'], q=10).apply(str)
    metrics['by_target_number'] = grouped_by_num.groupby('number_decile').agg({
        'correct_product' : [lambda x: pd.Series([np.mean([any(y) for y in x])]), 'size'],
        'correct_factorization' : [lambda x: pd.Series([np.mean([any(y) for y in x])]), 'size']
    })
    metrics['by_target_number'].columns = ['correct_product_mean', 'correct_product_size', 'correct_factorization_mean', 'correct_factorization_size']
    metrics['by_target_number'] = metrics['by_target_number'].to_dict()
    
    
    
    return metrics

In [ ]:
def save_json(path, to_save):
    with open(path, 'w') as f:
        json.dump(to_save, f, indent=4)

In [ ]:
metrics = compute_metrics(metric_df)

In [ ]:
save_json(save_path + 'metrics.json', metrics)

In [ ]:
metric_df.groupby('target_num').agg({'correct_product' : 'any'}).mean()

In [ ]:
np.log(1)

In [ ]:
metrics